In [1]:
pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 475.8 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymupdf4llm]
Note: you may need to restart the kernel to use updated packages.


In [18]:
ls ../../data/

RFP1.PDF  RFP2.pdf  RFP3.pdf


In [6]:
!ls ../../../

app.py	chroma_db  demo  __pycache__  requirements.txt
build	data	   logs  READMe.md    src


In [7]:
import pymupdf4llm


pdf_pth = "../../../data/RFP1.PDF"
fn = pdf_pth.lower().split("/")[-1].strip(".pdf")
md_text = pymupdf4llm.to_markdown(pdf_pth)
md_text

"**Respondent Q&A Zoom** : Tuesday April 1, 2025 9am\n\n\n=\n[https://us02web.zoom.us/j/85361672810?pwd](https://us02web.zoom.us/j/85361672810?pwd=cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1) cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1\n\n\n**RFP General Information contact** [: LLR-Support@bedford2030.org](mailto:LLR-Support@bedford2030.org) or call (646) 389-6749\n\n\n**Submission via email only** : townclerk@bedfordny.gov\n\n\n________________________________________________________________\n\n\n**I. PURPOSE**\n\n\nThis Request for Proposal (RFP) seeks responses from qualified NYSERDA-approved third-party\nenergy auditors to complete health and safety energy audits on multi-family rental buildings in the Town\nof Bedford (the “Town”). Successful respondents will be included in a Town-approved list from which\nlandlords will select and engage directly with the Town of Bedford Building Inspector and other Town\npersonnel to perform inspections of certain multi-family rental residences located in the Town. 

In [8]:
fn

'rfp1'

In [9]:
# now work with the markdown text, e.g. store as a UTF8-encoded file
import pathlib
pathlib.Path(f"{fn}.md").write_bytes(md_text.encode())

16532

# function for heading to heading segregation
**in test cases, i found** 
heading are either can be detected:  
- bullet point and all captilize letters or 
- start with  ##


In [53]:
import re
import pymupdf4llm


def has_alpha(line: str) -> bool:
    return bool(re.search(r"[A-Za-z]", line))


def is_heading(line: str) -> bool:
    """Detect if a line is a heading."""
    line = line.strip()
    # Markdown heading
    if line.startswith("## ") or line.startswith("# "):
        return True
    # Bold uppercase heading
    # bold_match = re.match(r"\*\*(.+?)\*\*", line)
    # if bold_match and bold_match.group(1).replace(" ", "").isupper():
    #     return True
    return False


def split_by_headings(markdown_text: str) -> list[str]:
    """
    Split markdown text into sections starting with a heading.
    If a heading has no content, it is concatenated to the next section.
    """
    lines = markdown_text.splitlines()
    sections = []
    current_heading = None
    current_content = []

    for line in lines:
        line = line.strip()

        if not line or not has_alpha(line):
            continue

        if is_heading(line):

            # If previous heading has content, save it
            if current_heading and current_content:
                sections.append(current_heading + "\n" + "\n".join(current_content))
                current_content = []
                current_heading = line

            # If previous heading had no content, carry it forward
            elif current_heading and not current_content:
                # Concatenate repeated heading
                current_heading += "\n" + line

            else:
                current_heading = line

        else:
            if current_heading:
                current_content.append(line)
            else:
                # Content without heading
                current_heading = ""
                current_content.append(line)

    # Add last section
    if current_heading:
        sections.append(current_heading + "\n" + "\n".join(current_content))

    return sections


pdf_pth = "../../../data/RFP3.pdf"
fn = pdf_pth.lower().split("/")[-1].strip(".pdf")
md_text = pymupdf4llm.to_markdown(pdf_pth)
sections = split_by_headings(md_text)
print(sections)

['## **Edgemont Union Free School District**\n**Scarsdale, New York**', '# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**\n#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**\n### May 20, 2025\n(Publication Date)\n### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**\n#### **at the Office of the Director of Facilities\u200b** **Return Proposals to:** **Edgemont Union Free School District\u200b** **℅ Rosario Renda, Director of Facilities\u200b** **Facilities Department\u200b** **300 White Oak Lane\u200b** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org', '## **Edgemont Union Free School District**\n**Scarsdale, New York**\n**NOTICE TO PROPOSERS**\n**CONSTRUCTION INSPECTION & TESTING AGENCIES**\nThe Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals\n(RFP) for the selection of a construction inspection and testing 

In [54]:
for section in sections:
    print(len(section.split(" ")))

8
70
181
256
283
306
319
315
113
402
94
47
265
560
127
291
334
491
75
93
137
100
51
8
135


**merge shorter sections**

In [55]:
min_tokens = 50
merge_sections = []
prev_section = ""

for curr_section in sections:
    
    if prev_section:
        # update the current section by concatenate with prev section
        curr_section = f"{prev_section}\n{curr_section}"
    
    # if tokens shorter than thresh: update prev section with current
    curr_tokens = len(curr_section.split(" ")) 
    if curr_tokens < min_tokens:
        prev_section = curr_section
    
    # if tokens are normal just append it merge section
    else:
        merge_sections.append(curr_section)
        prev_section = ""

if prev_section:
    if merge_sections:
        merge_sections[-1] += f"\n{prev_section}"
    else:
        merge_sections.append(prev_section)

merge_sections

['## **Edgemont Union Free School District**\n**Scarsdale, New York**\n# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**\n#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**\n### May 20, 2025\n(Publication Date)\n### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**\n#### **at the Office of the Director of Facilities\u200b** **Return Proposals to:** **Edgemont Union Free School District\u200b** **℅ Rosario Renda, Director of Facilities\u200b** **Facilities Department\u200b** **300 White Oak Lane\u200b** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org',
 '## **Edgemont Union Free School District**\n**Scarsdale, New York**\n**NOTICE TO PROPOSERS**\n**CONSTRUCTION INSPECTION & TESTING AGENCIES**\nThe Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals\n(RFP) for the selection of a construction inspection and testing a

In [56]:
for section in merge_sections:
    print(len(section.split(" ")))

77
181
256
283
306
319
315
113
402
94
311
560
127
291
334
491
75
93
137
100
51
142


In [57]:
sections = ["ae 121"]
min_tokens = 3
merge_sections = []
prev_section = ""
for curr_section in sections:
    
    if prev_section:
        # update the current section by concatenate with prev section
        curr_section = f"{prev_section}\n{curr_section}"
    
    # if tokens shorter than thresh: update prev section with current
    curr_tokens = len(curr_section.split(" ")) 
    if curr_tokens < min_tokens:
        prev_section = curr_section
    
    # if tokens are normal just append it merge section
    else:
        merge_sections.append(curr_section)
        prev_section = ""

if prev_section:
    if merge_sections:
        merge_sections[-1] += f"\n{prev_section}"
    else:
        merge_sections.append(prev_section)

merge_sections

['ae 121']

In [37]:
for i, curr_section in enumerate(sections):
    curr_tokens = len(curr_section.split(" "))
    if i > 0 and curr_tokens < max_tokens:
        prev_section = merge_chunks[i-1]
        merge_chunks[i-1] = f"{prev_section}\n{curr_section}" 
    else:
        merge_chunks.append(curr_section)

IndexError: list index out of range

In [6]:
pdf_pth = "../../data/RFP2.pdf"
fn = pdf_pth.lower().split("/")[-1].strip(".pdf")
md_text = pymupdf4llm.to_markdown(pdf_pth)
sections = split_by_headings(md_text)
print(sections)

['**REQUEST FOR PROPOSALS**\n**PROPOSAL INSTRUCTIONS AND CONDITIONS**\n**FOR**\n**CONSTRUCTION MANAGEMENT SERVICES**\n**FOR THE**\n**EAST ISLIP UNION FREE SCHOOL DISTRICT**\nThe Board of Education, East Islip Union Free School District is issuing this Request for Proposals\n(RFP) for the selection of a construction management firm to provide construction management\nservices in connection with various capital improvements to be undertaken by the School District\nduring the 2025-2026 school year and such other projects as the School District may request. The\nSchool District has established a capital reserve for capital improvements at the District’s\nfacilities. The School District anticipates that an annual amount of up to $3,000,000 will be\ndeposited into the capital reserve for use in connection with capital projects subject to voter\napproval.\nThe School District is seeking one or more qualified individuals/firms to provide construction\nmanagement services in connection with the

In [7]:
pdf_pth = "../../data/RFP3.pdf"
fn = pdf_pth.lower().split("/")[-1].strip(".pdf")
md_text = pymupdf4llm.to_markdown(pdf_pth)
sections = split_by_headings(md_text)
print(sections)

['## **Edgemont Union Free School District**\n**Scarsdale, New York**\n# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**\n#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**\n### May 20, 2025\n(Publication Date)\n### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**\n#### **at the Office of the Director of Facilities\u200b** **Return Proposals to:** **Edgemont Union Free School District\u200b** **℅ Rosario Renda, Director of Facilities\u200b** **Facilities Department\u200b** **300 White Oak Lane\u200b** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org', '## **Edgemont Union Free School District**\n**Scarsdale, New York**', '**NOTICE TO PROPOSERS**\n**CONSTRUCTION INSPECTION & TESTING AGENCIES**\nThe Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals\n(RFP) for the selection of a construction inspection and testing 

# extract metadata

In [50]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [40]:
from pydantic import BaseModel, Field

class RFPMetadata(BaseModel):
    company: str = Field(description="Name of organization or company")
    project: str = Field(description="Name of project")


In [47]:
extract_metadata_prompt = """Given RFP first page content, your task is to extract required information from it. If any of information missing, keep it empty.
RFP content:
{first_page_content}"""

structured_model = model.with_structured_output(RFPMetadata)
structured_model

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x794e87eaa3c0>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'RFPMetadata', 'description': '', 'parameters': {'properties': {'company': {'description': 'Name of organization or company', 'type': 'string'}, 'project': {'description': 'Name of project', 'type': 'string'}}, 'required': ['company', 'project'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'RFPMetadata', 'description': '', 'parameters': {'properties': {'company': {'description': 'Name of organization or company', 'type': 'string'}, 'project': {'description': 'Name of project', 'type': 'string'}}, 'required': ['c

In [66]:
from langchain_core.messages import HumanMessage

# System message
message = extract_metadata_prompt.format(
    first_page_content=md_text[0]['text'], 
    )

# Generate question 
rfp_metadata = structured_model.invoke([
        HumanMessage(content=message)
    ]
)

In [67]:
rfp_metadata, rfp_metadata.model_dump()

(RFPMetadata(company='Edgemont Union Free School District', project='Controlled Testing & Inspections for District Wide Additions & Alterations'),
 {'company': 'Edgemont Union Free School District',
  'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations'})

# recurssive chunking

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=['#', '##', '###', '####', '####', "\n\n", "\n", " ", ""])
texts = text_splitter.split_text(sections[0])

In [22]:
texts

['## **Edgemont Union Free School District**\n**Scarsdale, New York**\n# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**\n#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**\n### May 20, 2025\n(Publication Date)\n### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**\n#### **at the Office of the Director of Facilities\u200b** **Return Proposals to:** **Edgemont Union Free School District\u200b** **℅ Rosario Renda, Director of Facilities\u200b** **Facilities Department\u200b** **300 White Oak Lane\u200b** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org']

In [28]:
doc_chunks = []
for section in sections:
    doc_chunks.extend(text_splitter.split_text(section))

In [29]:
print("heading to heading sections: ", len(sections))
print("chunks sections: ", len(doc_chunks))

heading to heading sections:  38
chunks sections:  71


In [30]:
doc_chunks

['## **Edgemont Union Free School District**\n**Scarsdale, New York**\n# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**\n#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**\n### May 20, 2025\n(Publication Date)\n### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**\n#### **at the Office of the Director of Facilities\u200b** **Return Proposals to:** **Edgemont Union Free School District\u200b** **℅ Rosario Renda, Director of Facilities\u200b** **Facilities Department\u200b** **300 White Oak Lane\u200b** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org',
 '## **Edgemont Union Free School District**\n**Scarsdale, New York**',
 '**NOTICE TO PROPOSERS**\n**CONSTRUCTION INSPECTION & TESTING AGENCIES**\nThe Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals\n(RFP) for the selection of a construction inspection and testin

In [ ]:
# pip install strip-markdown

Note: you may need to restart the kernel to use updated packages.


In [41]:
import strip_markdown

def rm_markdown(md_text:str):
    return strip_markdown.strip_markdown(md_text)

doc_chunks = []
for section in sections:
    doc_chunks.extend(map(rm_markdown, text_splitter.split_text(section)))

In [36]:
doc_chunks

[]

In [37]:
doc_chunks = []
for section in sections:
    doc_chunks.extend(text_splitter.create_documents(section))

In [45]:
section_chunks = text_splitter.create_documents(sections)
# strip mardown format and add metadata

metadata = {
    'company': 'Edgemont Union Free School District',
    'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations',
    'file_name': fn
    }

for i in range(len(section_chunks)):
    section_chunks[i].page_content = rm_markdown(section_chunks[i].page_content)
    section_chunks[i].metadata = metadata

In [46]:
section_chunks

[Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'rfp3'}, page_content='Edgemont Union Free School District\nScarsdale, New York\nREQUEST FOR PROPOSALS CONTROLLED TESTING & INSPECTIONS FOR DISTRICT WIDE ADDITIONS & ALTERATIONS\n(RFP-2025-BO-003-Controlled Testing & Inspections)\nMay 20, 2025\n(Publication Date)\nDeadline for Proposals: Thursday, May 29, 2025, at 11:00 a.m.\nat the Office of the Director of Facilities\u200b Return Proposals to: Edgemont Union Free School District\u200b ℅ Rosario Renda, Director of Facilities\u200b Facilities Department\u200b 300 White Oak Lane\u200b Scarsdale, New York 10583 Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org'),
 Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'rfp3'}, page_content='Edgemont Unio

# embedding creation

In [53]:
from langchain_community.embeddings import JinaEmbeddings

_set_env("JINA_API_KEY")
text_embeddings = JinaEmbeddings(
    model_name="jina-embeddings-v3"
)

In [67]:
text = "faizan is male"
query_result = text_embeddings.embed_query(text)
len(query_result)

1024

In [68]:
import numpy as np
e1 = np.array(query_result).reshape(1, -1)
e1.shape

(1, 1024)

In [69]:
text = "faizan belong male gender"
e2 =  np.array(text_embeddings.embed_query(text)).reshape(-1, 1)

In [70]:
e2.shape

(1024, 1)

In [71]:
cosine_sim = np.dot(e1, e2)
cosine_sim

array([[0.97018824]])

In [72]:
section_chunks

[Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'rfp3'}, page_content='Edgemont Union Free School District\nScarsdale, New York\nREQUEST FOR PROPOSALS CONTROLLED TESTING & INSPECTIONS FOR DISTRICT WIDE ADDITIONS & ALTERATIONS\n(RFP-2025-BO-003-Controlled Testing & Inspections)\nMay 20, 2025\n(Publication Date)\nDeadline for Proposals: Thursday, May 29, 2025, at 11:00 a.m.\nat the Office of the Director of Facilities\u200b Return Proposals to: Edgemont Union Free School District\u200b ℅ Rosario Renda, Director of Facilities\u200b Facilities Department\u200b 300 White Oak Lane\u200b Scarsdale, New York 10583 Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org'),
 Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'rfp3'}, page_content='Edgemont Unio

## vector db creation

In [ ]:
from langchain_community.vectorstores import Chroma

DB_PERSIST_DIRECTORY = "./chroma_db"



Chroma.from_documents(
            documents=section_chunks,
            embedding=text_embeddings,
            persist_directory=DB_PERSIST_DIRECTORY,
        )

In [78]:
ls

chroma_db/  chunkings.ipynb  output.md  rfp1.md  rfp2.md  rfp3.md  test.py


# Markdown chunking

In [8]:
sections

['## **Edgemont Union Free School District**\n**Scarsdale, New York**\n# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**\n#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**\n### May 20, 2025\n(Publication Date)\n### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**\n#### **at the Office of the Director of Facilities\u200b** **Return Proposals to:** **Edgemont Union Free School District\u200b** **℅ Rosario Renda, Director of Facilities\u200b** **Facilities Department\u200b** **300 White Oak Lane\u200b** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org',
 '## **Edgemont Union Free School District**\n**Scarsdale, New York**',
 '**NOTICE TO PROPOSERS**\n**CONSTRUCTION INSPECTION & TESTING AGENCIES**\nThe Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals\n(RFP) for the selection of a construction inspection and testin

In [11]:
print(sections[0])

## **Edgemont Union Free School District**
**Scarsdale, New York**
# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**
#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**
### May 20, 2025
(Publication Date)
### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**
#### **at the Office of the Director of Facilities​** **Return Proposals to:** **Edgemont Union Free School District​** **℅ Rosario Renda, Director of Facilities​** **Facilities Department​** **300 White Oak Lane​** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org


In [14]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

markdown_document = "# Foo\n\n    ## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n ### Boo \n\n Hi this is Lance \n\n ## Baz\n\n Hi this is Molly"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(sections[0])
md_header_splits

[Document(metadata={'Header 2': '**Edgemont Union Free School District**'}, page_content='**Scarsdale, New York**'),
 Document(metadata={'Header 1': '**REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**'}, page_content='#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**'),
 Document(metadata={'Header 1': '**REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**', 'Header 3': 'May 20, 2025'}, page_content='(Publication Date)'),
 Document(metadata={'Header 1': '**REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**', 'Header 3': '**Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**'}, page_content='#### **at the Office of the Director of Facilities** **Return Proposals to:** **Edgemont Union Free School District** **℅ Rosario Renda, Director of Facilities** **Facilities Department** **300 Wh

In [58]:
docs[0]

Document(metadata={'source': './rfp1.md', 'emphasized_text_contents': ['Respondent Q&A Zoom'], 'emphasized_text_tags': ['b'], 'languages': ['eng'], 'file_directory': '.', 'filename': 'rfp1.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-22T15:00:45', 'category': 'UncategorizedText', 'element_id': '2c056321cc60f084a0817352c44d4263'}, page_content='Respondent Q&A Zoom : Tuesday April 1, 2025 9am')

In [59]:
docs

[Document(metadata={'source': './rfp1.md', 'emphasized_text_contents': ['Respondent Q&A Zoom'], 'emphasized_text_tags': ['b'], 'languages': ['eng'], 'file_directory': '.', 'filename': 'rfp1.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-22T15:00:45', 'category': 'UncategorizedText', 'element_id': '2c056321cc60f084a0817352c44d4263'}, page_content='Respondent Q&A Zoom : Tuesday April 1, 2025 9am'),
 Document(metadata={'source': './rfp1.md', 'link_texts': ['https://us02web.zoom.us/j/85361672810?pwd'], 'link_urls': ['https://us02web.zoom.us/j/85361672810?pwd=cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1'], 'languages': ['eng'], 'file_directory': '.', 'filename': 'rfp1.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-22T15:00:45', 'category': 'UncategorizedText', 'element_id': 'ba774ad6388395c4ba07652ac6b59699'}, page_content='= https://us02web.zoom.us/j/85361672810?pwd cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1'),
 Document(metadata={'source': './rfp1.md', 'emphasized_text_contents': ['RF

# test final

In [58]:
import sys


sys.path.append("../../../")

In [59]:
sys.path

['/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python313.zip',
 '/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python3.13',
 '/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python3.13/lib-dynload',
 '',
 '/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python3.13/site-packages',
 '../../../']

In [60]:
ls

chroma_db/  chunkings.ipynb  documents.json  logs/  rfp1.md  simple_rag.ipynb


In [3]:
from src.indexing.utils import (
    split_by_headings,
    get_pdf_markdown,
    rm_markdown,
    extract_rfp_metadata,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter


def chunk_h2h_sections(h2h_sections: list[str], file_name: str) -> list:
    rfp_metadata = extract_rfp_metadata(h2h_sections[0])
    if isinstance(rfp_metadata, dict):
        rfp_metadata["file_name"] = file_name
    else:
        rfp_metadata = {"file_name": file_name}

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["#", "##", "###", "####", "####", "\n\n", "\n", " ", ""],
    )

    chunks = text_splitter.create_documents(h2h_sections)
    for i in range(len(chunks)):
        chunks[i].page_content = rm_markdown(chunks[i].page_content)
        chunks[i].metadata = rfp_metadata
    return chunks


pdfs_path = "data/"
pdf_pth = "../data/RFP1.PDF"
fn = pdf_pth.lower().split("/")[-1].strip(".pdf")
md_text = get_pdf_markdown(pdf_pth)
h2h_sections = split_by_headings(md_text)
chunks = chunk_h2h_sections(h2h_sections, fn)
print(f"Total Chunks Created: {len(chunks)}")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.
Total Chunks Created: 21


In [4]:
chunks

[Document(metadata={'company': 'Town of Bedford', 'project': 'Health and safety energy audits on multi-family rental buildings', 'file_name': 'rfp1'}, page_content='I. PURPOSE\nRespondent Q&A Zoom : Tuesday April 1, 2025 9am\nhttps://us02web.zoom.us/j/85361672810?pwd cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1\nRFP General Information contact : LLR-Support@bedford2030.org or call (646) 389-6749\nSubmission via email only : townclerk@bedfordny.gov\nThis Request for Proposal (RFP) seeks responses from qualified NYSERDA-approved third-party\nenergy auditors to complete health and safety energy audits on multi-family rental buildings in the Town\nof Bedford (the “Town”). Successful respondents will be included in a Town-approved list from which\nlandlords will select and engage directly with the Town of Bedford Building Inspector and other Town\npersonnel to perform inspections of certain multi-family rental residences located in the Town. These'),
 Document(metadata={'company': 'Town of Bedford', 'p

In [7]:
import os

file_path = "/usr/local/data/report.csv"
file_name = os.path.basename(file_path)
print(f"File name with extension: {file_name}")


File name with extension: report.csv


In [5]:
from src.indexing.ingest import chunk_all_pdfs

pdfs_path = "../data/"
all_chunks = chunk_all_pdfs(pdfs_path)
print(f"Total Chunks Created from all PDFs: {len(all_chunks)}")

Total Chunks Created from all PDFs: 132


In [6]:
all_chunks

[Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'RFP3.pdf'}, page_content='Edgemont Union Free School District\nScarsdale, New York\nREQUEST FOR PROPOSALS CONTROLLED TESTING & INSPECTIONS FOR DISTRICT WIDE ADDITIONS & ALTERATIONS\n(RFP-2025-BO-003-Controlled Testing & Inspections)\nMay 20, 2025\n(Publication Date)\nDeadline for Proposals: Thursday, May 29, 2025, at 11:00 a.m.\nat the Office of the Director of Facilities\u200b Return Proposals to: Edgemont Union Free School District\u200b ℅ Rosario Renda, Director of Facilities\u200b Facilities Department\u200b 300 White Oak Lane\u200b Scarsdale, New York 10583 Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org'),
 Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'RFP3.pdf'}, page_content='Edgem

In [1]:
import sys


sys.path.append("../../../")

In [ ]:
from src.indexing.ingest import ingest_data


ingest_data()

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [63]:
sys.path

['/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python313.zip',
 '/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python3.13',
 '/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python3.13/lib-dynload',
 '',
 '/home/muhammad-faizan/miniconda3/envs/langsmith-env/lib/python3.13/site-packages',
 '../../../',
 '/tmp/tmpo2lsvlu5',
 '../../../']

In [67]:
!ls ../../../

app.py	chroma_db  demo  __pycache__  requirements.txt
build	data	   logs  READMe.md    src


In [2]:
from src.indexing.ingest import chunk_pdf


chunks = chunk_pdf("../../../data/RFP3.pdf")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [3]:
len(chunks)

36

In [4]:
for chunk in chunks:
    print(len(chunk.page_content.split()))
    print(chunk.page_content)
    print("---"*50)

78
Edgemont Union Free School District
Scarsdale, New York
REQUEST FOR PROPOSALS CONTROLLED TESTING & INSPECTIONS FOR DISTRICT WIDE ADDITIONS & ALTERATIONS
(RFP-2025-BO-003-Controlled Testing & Inspections)
May 20, 2025
(Publication Date)
Deadline for Proposals: Thursday, May 29, 2025, at 11:00 a.m.
at the Office of the Director of Facilities​ Return Proposals to: Edgemont Union Free School District​ ℅ Rosario Renda, Director of Facilities​ Facilities Department​ 300 White Oak Lane​ Scarsdale, New York 10583 Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org
------------------------------------------------------------------------------------------------------------------------------------------------------
196
Edgemont Union Free School District
Scarsdale, New York
NOTICE TO PROPOSERS
CONSTRUCTION INSPECTION & TESTING AGENCIES
The Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals
(RFP) for the selection of a construction inspection and 

In [75]:
chunks

[Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'RFP3.pdf'}, page_content='Edgemont Union Free School District\nScarsdale, New York\nREQUEST FOR PROPOSALS CONTROLLED TESTING & INSPECTIONS FOR DISTRICT WIDE ADDITIONS & ALTERATIONS\n(RFP-2025-BO-003-Controlled Testing & Inspections)\nMay 20, 2025\n(Publication Date)\nDeadline for Proposals: Thursday, May 29, 2025, at 11:00 a.m.\nat the Office of the Director of Facilities\u200b Return Proposals to: Edgemont Union Free School District\u200b ℅ Rosario Renda, Director of Facilities\u200b Facilities Department\u200b 300 White Oak Lane\u200b Scarsdale, New York 10583 Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org'),
 Document(metadata={'company': 'Edgemont Union Free School District', 'project': 'Controlled Testing & Inspections for District Wide Additions & Alterations', 'file_name': 'RFP3.pdf'}, page_content=''),
 

In [76]:
len(chunks)

67

In [16]:
import sys


sys.path.append("../../../")

from src.indexing.ingest import chunk_h2h_sections, get_recurrsive_chunks, create_finalize_chunks
from src.indexing.utils import merge_shorter_sections, get_pdf_markdown, split_by_headings
import os

pdf_path = "../../../data/RFP1.PDF"
md_text = get_pdf_markdown(pdf_path)
h2h_sections = split_by_headings(md_text)
if not h2h_sections:
    print(f"No headings found for {pdf_path}")



In [17]:
merged_sections = merge_shorter_sections(h2h_sections)
print(f"Total Chunks Created from {pdf_path}: {len(merged_sections)}")

Total Chunks Created from ../../../data/RFP1.PDF: 5


In [18]:
chunks = create_finalize_chunks(merged_sections, file_name="RFP3.pdf")

In [19]:
chunks

[Document(metadata={'company': 'Town of Bedford', 'project': 'Health and safety energy audits on multi-family rental buildings', 'file_name': 'RFP3.pdf'}, page_content='I. PURPOSE\nRespondent Q&A Zoom : Tuesday April 1, 2025 9am\nhttps://us02web.zoom.us/j/85361672810?pwd cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1\nRFP General Information contact : LLR-Support@bedford2030.org or call (646) 389-6749\nSubmission via email only : townclerk@bedfordny.gov\nThis Request for Proposal (RFP) seeks responses from qualified NYSERDA-approved third-party\nenergy auditors to complete health and safety energy audits on multi-family rental buildings in the Town\nof Bedford (the “Town”). Successful respondents will be included in a Town-approved list from which\nlandlords will select and engage directly with the Town of Bedford Building Inspector and other Town\npersonnel to perform inspections of certain multi-family rental residences located in the Town. These\naudits are required by the Town’s Landlord Registr

In [6]:
len(chunks)

21

In [20]:
for i, chunk in enumerate(chunks):
    print(f"chunk-{i+1}")
    print("total tokens: ", len(chunk.page_content.split()))
    print(" \n\ntext:")
    print(chunk.page_content)
    print("------------------------------------------------------------------\n")

chunk-1
total tokens:  314
 

text:
I. PURPOSE
Respondent Q&A Zoom : Tuesday April 1, 2025 9am
https://us02web.zoom.us/j/85361672810?pwd cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1
RFP General Information contact : LLR-Support@bedford2030.org or call (646) 389-6749
Submission via email only : townclerk@bedfordny.gov
This Request for Proposal (RFP) seeks responses from qualified NYSERDA-approved third-party
energy auditors to complete health and safety energy audits on multi-family rental buildings in the Town
of Bedford (the “Town”). Successful respondents will be included in a Town-approved list from which
landlords will select and engage directly with the Town of Bedford Building Inspector and other Town
personnel to perform inspections of certain multi-family rental residences located in the Town. These
audits are required by the Town’s Landlord Registry and Rental Property Health and Performance Law
(the “Law”) and must be conducted on qualified registered rental buildings (defined as 2+ unit

In [ ]:
import sys


sys.path.append("../../../")

from src.indexing.ingest import chunk_pdf


chunks = get_pdf_markdown("../../../data/RFP3.pdf")

# remove heading - footer

In [2]:
# pip install pymupdf-layout

In [1]:
# import pymupdf.layout
# import pymupdf4llm


# pdf_path = "../../../data/RFP3.pdf"
# md_text = pymupdf4llm.to_markdown(pdf_path, header=False, footer=False)

In [3]:
# md_text

In [ ]:
import pymupdf4llm
from IPython.display import Markdown


def filter_header_footer(lines: list[str], line_counts:dict[str, int], count_threshold:int):
    # Filter out lines appearing more than threshold times
    filtered_lines = []
    for line in lines:
        stripped = line.strip()
        if line_counts.get(stripped, 0) <= count_threshold:
            filtered_lines.append(line)
    return "\n".join(filtered_lines)


def get_line_counts(lines, max_chars=100):
    """header footer are usually short in length, not more than `max_chars` threshold"""
    line_counts = {}
    for line in lines:
        stripped = line.strip()
        if stripped and len(stripped) < max_chars:  # Headers/footers are usually short
            line_counts[stripped] = line_counts.get(stripped, 0) + 1
    
    print(f"top five line counts: {dict(sorted(line_counts.items(), key=lambda item: item[1], reverse=True)[:5])}")
    return line_counts

def remove_repetitive_headers_footers(markdown_text, threshold=8):
    """Remove lines that appear repeatedly (likely headers/footers)."""
    try:
        lines = markdown_text.splitlines()
        line_counts = get_line_counts(lines)
        filtered_text = filter_header_footer(lines, line_counts, threshold)
        return filtered_text
    except Exception as e:
        print(f"Failed to remove the header footer text due to: {e}")
        return ""

cleaned_md = remove_repetitive_headers_footers(markdown_content)

top five line counts: {'## **Edgemont Union Free School District Scarsdale, New York**': 12, '## **Scarsdale, New York**': 11, '## **Edgemont Union Free School District**': 10, '||Full Day||||': 10, '|---|---|---|---|---|': 5}


In [ ]:
pdfs = "../../../data"
import os

files = os.listdir(pdfs)
for file in files:
    md_text = pymupdf4llm.to_markdown(os.path.join(pdfs,file))
    display(Markdown(f"# File: {file}"))
    cleaned_md = remove_repetitive_headers_footers(md_text)
    display(Markdown(cleaned_md))


# File: RFP3.pdf

top five line counts: {'## **Edgemont Union Free School District**': 24, '**Scarsdale, New York**': 24, '||Full Day||||': 11, '|---|---|---|---|---|': 6, '|Formal Reports / Daily|Formal Reports / Daily||ea||': 6}




# **REQUEST FOR PROPOSALS** **CONTROLLED TESTING & INSPECTIONS FOR** **DISTRICT WIDE ADDITIONS &** **ALTERATIONS**

#### **(RFP-2025-BO-003-Controlled Testing & Inspections)**

### May 20, 2025


(Publication Date)

### **Deadline for Proposals:** **Thursday, May 29, 2025, at 11:00 a.m.**

#### **at the Office of the Director of Facilities​** **Return Proposals to:** **Edgemont Union Free School District​** **℅ Rosario Renda, Director of Facilities​** **Facilities Department​** **300 White Oak Lane​** **Scarsdale, New York 10583** Phone (914) 725-1500 ext. 4418 rrenda@edgemont.org





**NOTICE TO PROPOSERS**


**CONSTRUCTION INSPECTION & TESTING AGENCIES**


The Board of Education of the Edgemont Union Free School District is issuing this Request for Proposals
(RFP) for the selection of a construction inspection and testing agency in connection with certain capital
improvement projects to be undertaken within the Edgemont Union Free School District’s facilities at
Seely Elementary School, Greenville Elementary School, and Edgemont Junior/Senior High School.


The successful agency will be required to perform all necessary inspections and tests required in
connection with the capital improvements including inspections and testing required by laws, ordinances,
rules, regulations, or order of public authorities, the General Conditions of the Contract for Construction
and the plans and specifications for each project. You are invited to submit proposals in accordance with

this RFP.


Proposals will be received until May 29, 2025, at 11:00 a.m. at the Edgemont Union Free School District
300 White Oak Lane Scarsdale, NY 10583 Attention Ray Renda, Director of Facilities. Proposal forms
may be obtained at the aforementioned office. The Edgemont Union Free School District reserves the
right to reject any proposal or to accept part of any proposal.





**A.​** **ABOUT THIS RFP:**

The Edgemont Union Free School District requests a proposal for unit cost pricing to perform
Controlled testing and Inspection Services for the capital improvements projects listed below:


1.​ Edgemont Junior/ Senior High School - Addition and Alterations Project
2.​ Greenville Elementary School - Alterations Project
3.​ Seely Elementary School - Alterations Project


**Special Inspection Agency Requirements**


The Special Inspection Agency must be approved and meet all applicable requirements of the NYS
Department of State Division of Code Enforcement Administration (hereafter referred to as
NYSDSDCEA). All equipment used must be adequate to perform required tests and all employed
personnel must be experienced and educated in conducting, supervising, and evaluating tests and/or
inspections. Please review the attached Special Inspector Minimum Qualifications prior to submitting a
proposal. Proof of qualifications must be submitted to the Owner as part of the proposal.


All inspections must be documented with reports indicating that all work was done in conformance to
approved construction documents, and be furnished to the Code Enforcement Official, Owner, its
designee, and Architect. Any discrepancies must be documented appropriately and reported, as well as
corrections must also be documented and reported to the Code Enforcement Official, Owner, its
designee, and Architect.


**B.​ SCHEDULE**


Construction activities of the District Wide Additions and Alteration Projects are anticipated to start
and complete as listed below:


1.​ Seely Elementary School - Addition and Alterations Project 
a.​ Design Start 6/21/25, Anticipated Construction Start Date 7/1/2026 or sooner
b.​ Construction Completion 9/1/27.
2.​ Greenville Elementary School - Alterations Project 

a.​ Construction Start 6/1/25

b.​ Construction Completion 9/1/27.
3.​ Edgemont Junior/Senior High School - Alterations Project 

a.​ Construction Start 6/1/25

b.​ Construction Completion 6/1/28.





**C.​SCOPE OF SERVICES:**


The successful proposer shall provide the Edgemont Union Free School District with construction,
inspection and testing services in connection with the capital improvement projects set forth below. ​


1.​ Seely Elementary School – Alterations Project
2.​ Greenville Elementary School – Alterations Project
3.​ Edgemont Junior/ Senior High School – Addition and Alterations Project


Required testing services will include but not be limited to soil testing and compaction inspections,
concrete and rebar testing and inspections, miscellaneous and structural steel inspections, masonry
and mortar testing and inspections, asphalt and concrete sitework inspections.


Responsibilities of the Multi-Discipline Inspector to include but not limited to:


1.​ Periodic inspections and testing in accordance with the contract drawings, plans,

specifications and as required by the Building Code of New York State, applicable code,
law, rule, regulations or order of public authority and as set forth in the General
Conditions of the Contract for Construction. All inspection activities are to be
supervised by a NY State Licensed Professional Engineer. Simultaneous inspections
shall be performed by the same inspector on a given day, i.e., compaction, rebar, and
concrete testing, certified in each discipline.​

2.​ Footing bottom inspections.​


3.​ Inspection, sampling and testing of soil, compaction/backfilling by a soil technician.​


4.​ Reinforcing steel, formwork, anchor bolt, and insert inspection by an ICC or ACI Concrete

Construction Specialist.​

5.​ Concrete testing (compressive strength, slump, temperature, air content, etc.) by an ACI

Field Testing technician - Grade I. Laboratory Concrete testing (compressive strength, etc.)
by an ACI Concrete Laboratory Technician.​

6.​ Fireproofing inspection - Density / adhesion /cohesion /thickness / review of product data


sheets.​

7.​ Masonry inspections by an ACI Masonry Field Technician for field verifications and ACI


Masonry.​

8.​ Laboratory Technician for laboratory tests:

a.​ Material verification / plumbness / proper placement procedures

b.​ Reinforcement verification

c.​ Mortar strength / grout compressive strength​





9.​ In-place testing of asphalt thickness, surface smoothness and density. Verify the height of

traffic-calming devices.​

10.​Structural Steel Inspections by the appropriate ICC certified Inspector (“Steel and Bolting”

or “Welding”). Multi- pass filet welds, penetration welds and other welds as called out by
the Architect or Engineer of Record shall be Non-Destructive Tested (NDT) by an AWS
Certified Welding Inspector:

a.​ Plumbness, alignment and erection.
b.​ High strength bolts/torque.
c.​ Welds, certifications

d.​ Review mill certifications and steel fabrication.​


11.​ Special Inspection Notes

a.​ Written reports are to be provided for each day’s work; signed and sealed by a NY

Licensed Professional Engineer. Written test and/or inspector reports including
actual readings, analysis, summary of findings and as applicable,
recommendations upon completion of each test or inspection phase of work.​

b.​ Copies of all special inspection reports shall be provided to the Architect and

Owner’s Representative/Construction Manager on a weekly basis. Any inspection/
report that indicates a failure or a deviation from the contract documents shall be
provided to the Architect and Owner’s representative immediately within (24
hours of the inspection).​

c.​ The testing agency shall be required to maintain a special inspection book. The

book shall be a three-ring binder which contains copies of all inspection reports.
At the completion of the project three (3) complete copies of the inspection report

book shall be turned over to the Owner.​

d.​ All inspection reports shall include the date of the inspection, the specific location

where the inspection was conducted and that the work inspected was done in
conformance with the construction documents or shall be corrected, along with all
other pertinent information.​

e.​ Where fabrication of a structural assembly is being performed on the premises of a

fabricators shop, special inspections shall be required.​

f.​ Grade and mill test reports are required for main stress carrying steel elements​

g.​ Prior to the placement of fill, the special inspector shall determine that the site has

been prepared in accordance with the contract documents.​





h.​ During fill placements and compaction, the special inspector shall determine that

the material being used, the maximum lift thickness and the in place dry density
comply with the contract documents.​

i.​ Special inspections are required for sprayed fire-resistant material being applied to
structural elements and decks, including thickness, density, and bond strength​

j.​ The School District has title to this property and the right of entry for these
inspections.


**D.​AVAILABILITY:**


1.​ The successful proposer shall have an inspector on site within one (1) day/24-hours’

notice. The Construction Manager and/or the Prime Contractor may coordinate site
visits with the testing laboratory.​

2.​ Cancellation of an inspection shall occur by 4:00PM the day prior to the scheduled

inspection. The School District shall not be responsible for payment of any
invoice for cancelled inspections or any cancellation fees. In the event of
inclement weather or other unforeseen circumstances that bars completion of or
occurrence of the inspection, the School District shall not incur fees associated
with cancellation or rescheduling. ​

3.​ Cancellation of an inspection (after inspector is on site) due to inclement weather,

will be billed for the time actually spent on site providing services.


**E.​MEETINGS**


The successful proposer must attend coordination meetings with the Owner, Architect, Structural
Engineer, Public Authorities, Construction Manager, and Contractors as may be necessary.


**F.​ INSURANCE REQUIREMENTS**


At the time of the proposal submission, the proposer will include a certificate of insurance. The
Proposer agrees to indemnify, defend and hold harmless the School District, Architect and
Construction Manager and its authorized officers, trustees, agents, and employees (the “Indemnities”)
from any claim, damage, liability, expense or loss including reasonable attorneys’ fees and other
costs, except for gross negligence or willful misconduct of the School District, Architect or
Construction Manager or its respective shareholders, officers, directors, agents, employees, or
consultants, or breach of this Agreement.


Without in any way affecting the indemnities herein provided and in addition thereto, the Proposer
shall secure and maintain throughout the Agreement the following types of insurance:


The policy naming the District as an additional insured shall:





a.​ Be an insurance policy from an A.M. Best A- rated or better insurer, licensed and admitted to

conduct business in New York State. A New York licensed and admitted insurer is required.


b.​ State that the organization’s coverage shall be primary and non-contributory coverage for the

District including a waiver of subrogation in favor of the District for all coverages including
Workers Compensation.


c.​ Additional insured status for General Liability coverage shall be provided by standard or other

endorsements that extended coverage to the District for on-going operations (CG 20 38 or
equivalent) and products and completed operations (CG 20 37 or equivalent). A completed copy
of the endorsements must be attached to the Certificate of Insurance to include General Liability,
Auto Liability, and Umbrella/Excess coverage.


d.​ At the District’s request, the contractor shall provide a copy of the declaration page of the liability

and umbrella/excess policies with a list of endorsements and forms. If requested, the contractor
will provide a copy of the policy endorsements and forms.


e.​ There will be no coverage restrictions and/or exclusions involving New York State Labor Law

statutes or gravity related injuries.


f.​ No policies containing escape clauses or exclusions contrary to the Owner’s interests will be
accepted.


g.​ Comprehensive General Liability Insurance:


a.​ $1,000,000 Each Occurrence & $2,000,000 General Aggregate
b.​ $2,000,000 Products and Completed Operations
c.​ $1,000,000 Personal and Advertising Injury
d.​ $100,000 Fire Damage
e.​ $10,000 Medical Expense
h.​ Owners Contractors Protective (OCP) Insurance

a.​ For projects greater than $1,000,000 and/or work over 1 story (10 feet);
b.​ $2,000,000 per Occurrence/$4,000,000 Aggregate with the District as the Named


Insured.

i.​ Automobile Liability Insurance:

a.​ $1,000,000 combined single limit for owned, hired, borrowed and non-owned motor


vehicles.

j.​ Workers’ Compensation Insurance.

a.​ Statutory Workers' Compensation (C-105.2 or U-26.3);
k.​ Professional Liability:

a.​ $1,000,000 Combined Single Limit.
l.​ Testing Company Errors & Omissions Insurance

a.​ $1,000,000 per Occurrence/$2,000,000 Aggregate
b.​ For inspection, testing and other professional acts of the contractor performed under


the contract with the School District.

m.​ Umbrella/Excess Liability:

a.​ $5,000,000 per Occurrence/$5,000,000 Aggregate.
**n.​** Waiver of Subrogation




All insurance policies, except Workers Compensation, shall contain primary and non-contributory
language, and additional insured endorsements naming the School District, Construction Manager,
Architects & Engineers as an ‘Additional Insured’.


**G.​ SUBMISSION OF PROPOSALS**


Respondents should submit an original and three (3) copies of their proposal. Proposals must be sealed
in manila envelopes, including the following title on the front of each package: **“PROPOSAL FOR**
**CONSTRUCTION TESTING AND INSPECTION SERVICES”** . Proposals must be sealed.
Proposals must be received no later than: **Thursday, May 29, 2025, 11:00 a.m.**


Edgemont Union Free School District​
℅ Rosario Renda, Director of Facilities​

Facilities Department​

300 White Oak Lane​

Scarsdale, New York 10583


PLEASE NOTE: ​

Proposals will not be publicly opened and read aloud.





**Right to Reject Proposals:**


This RFP does not commit the School District to award a contract, pay any cost incurred in the
preparation of a proposal in response to this RFP, or to procure or contract for services. The School
District intends to award a contract on the basis of the best interest and advantage to the School
District, and reserves the right to accept or reject any or all proposals received as a result of this RFP,
to negotiate with all qualified proposers, or to cancel this RFP in part or in its entirety, if it is in the
best interest of the School District to do so. No proposer shall have any legal, equitable, or
contractual rights of any kind arising out of its submission of a proposal except as and to the extent
that the School District, in its sole discretion, shall enter a contract with the proposer that it selects as

the successful vendor.


**Proposal Content:**


The purpose of the proposal is to demonstrate the qualifications, competence and capacity of the
Proposer seeking to provide the services of Construction Inspection and Testing. The proposal should
demonstrate the qualifications of the Proposer and of the staff to be assigned to this engagement.
While additional information may be presented, the following subjects listed below represent the
criteria against which the proposal will be evaluated and must be included:


1.​ Title Page showing the RFP’s subject; the Proposer’s name; the name, address, and telephone

number of a contact person; and the date of the proposal.​

2.​ Signed letter of transmittal briefly stating the proposer’s understanding of the work to be

done, the commitment to perform the work for the duration of the project(s), a statement
why the Proposer believes it to be best qualified to perform the service, submission of a
proposal shall be a firm offer to provide the requested services.​

3.​ A brief narrative detailing the Proposer’s history including location, size, and date of inception.​


4.​ Describe the Proposer’s experience and expertise in providing Construction Inspection and

Testing services to school districts.​

5.​ Names, resumes and copies of required licenses of personnel to be assigned to the school

district, including the partner(s) in charge. Briefly state the experience they have in
representing School Districts. It is fully expected that the personnel indicated will be those
assigned.​

6.​ Three (3) references from either other School Districts or governmental agencies, including

a contact name, title and phone number.​

7.​ An affirmation that there are no conflicts of interest between the Proposer and the School District.​





8.​ Disclosure and description of any disciplinary proceedings pending against the Proposer or

any of its employees in any jurisdiction or whether discipline has been previously imposed
against in any jurisdiction.​

9.​ Assurance of ability to provide executed copies of the Certificates of Insurances.​


10.​A completed Non-Collusive Certification​


11.​ A completed Non-Collusive Resolution (Required for Corporations)​


12.​A completed Proposer Warranties​


13.​A completed and Notarized Hold Harmless Agreement​


14.​A completed and Notarized Iran Divestment Act Certification​


15.​The completed Dollar Cost form- The Dollar Cost form will have Travel costs included in the unit

pricing.​

16.​Sexual Harassment Prevention Certification Form




**Information**


Questions concerning the RFP and the procedures for responding should be directed to:


William Means​

Fuller and D’Angelo Architects, PC ​

Tel.: 914-592-4444​

[williamm@fullerdangelo.com](mailto:williamm@fullerdangelo.com)


or


Kevin Sawyer ​

Triton Construction ​

Tel.: 845-821-3354​

[k-sawyer@tritonconstruction.net](mailto:k-sawyer@tritonconstruction.net)


with a copy to:


Rosario Renda​

Director of Facilities ​

Edgemont Union Free School District ​

Tel: 914-472-7767 ext. 4418​

[rrenda@edgemont.org](mailto:rrenda@edgemont.org)





**H.​ EVALUATION PROCEDURES**

These proposals are being solicited through a fair and open process. Applicants who are willing to
provide the services described as requested above, shall be evaluated based on experience and
qualifications as set forth in the criteria described below. For the selection of the finalist, the written
proposals will be reviewed based on the following:


1.​ General experience in the field.

2.​ References.

3.​ Qualifications and experience which most closely match the needs of the District.
4.​ Satisfaction with proposal requirements.
5.​ The competitiveness of applicant’s fee structure.


**I.​ CONTRACT DOCUMENTS PROVIDED:**


**Project Documents:**


It is encouraged that bidders obtain and review a copy of the Project’s Design Documentation from
the District’s Architect. Plans and Specifications may only be complete for a certain stage of the
project by the time of award for this contract. Fees are assumed to be based on standard unit prices
by the vendor, without prejudice to the design.


**Contract Period: ​**

The anticipated contract period will be defined in the Contract Agreement. Services for the projects
are expected to commence upon execution of the Agreement. The contract term may be renewed
annually for up to three (3) additional one-year terms.


The successful proposer shall be required to enter into a written Agreement with the School District
which will incorporate the terms of this RFP. If the successful proposer does not execute an
agreement within seven (7) days after being presented with one, the District reserves the right to
award the contract to the next qualified proposer.


**Attachments**


A.​ Non-Collusion Certification

B.​ Resolution - For Corporate Bidders Only
C.​ Proposers Warranties

D.​ Certification - Iran Divestment Act Of 2012

E.​ Hold Harmless Agreement
F.​ Proposal Form

G.​ Sexual Harassment Prevention Certification Form




#### **APPENDIX A** **NON-COLLUSIVE BIDDING/PROPOSAL CERTIFICATION**


(As required by Section 103d of the General Municipal Law as the act and deed of such corporation, and for any
inaccuracies or misstatements in such certificate, this corporation bidder shall be liable under the penalties of perjury).

As of September 1, 1966, this Section will read as follows
**103-d** Statement of non-collusion in bids and proposals to political subdivision of the state. Every bid or proposal
hereafter made to a political subdivision of the state or any public department, agency, or official thereof where
competitive bidding is required by statute, rule, regulation or local law, for work or services performed or to be
performed or goods sold or to be sold, shall contain the following statement subscribed by the bidders and affirmed by
such bidder as true under the penalties of perjury,

a.​ By submission of this bid, each bidder and each person signing on behalf of any bidder certifies, and in the case of
any joint bid each party thereto certified as to its own organization, under penalty of perjury, that to the best of
knowledge and belief:

1​ The prices in this bid have been arrived at independently without collusion, consultation, communication or
agreement, for the purpose of restricting competition, as to any matter relating to such prices with an other bidder
or with any competitor,
2​ Unless otherwise required by law, the prices which have been quoted in this bid have not been knowingly
disclosed by the bidder and will not knowingly be disclosed by the bidder prior to opening, directly or indirectly,
to any other bidder or to any competitor; and
3​ No attempt has been made or will be made by the bidder to induce any other person, partnership or corporation to
submit or not to submit a bid for the purpose of restricting competition.

b.​ Any bid shall not be considered for award nor shall any award be made where (a) (1) (2) and (3) above have not
been complied with; provided however, that if in any case the bidder shall so state and furnish with the bid a signed
statement which sets forth in detail the reason therefor. Where (a) (1) (2) and (3) above have not been complied with,
the bid shall not be considered for award nor shall any award be made unless the head of the purchasing unit of the
political subdivision, public department, agency or official thereof to which the bid is made, or his designee,
determines that such disclosure was not made for the purpose of restricting competition.

The fact that the bidder (a) has published price lists, rates, or tariffs covering items being procured, (b) has informed
prospective customers of proposed or pending publication of new or revised price lists for such items, or (c) has sold the
same items to other customers at the same prices being bid, does not constitute, without more, a disclosure within the
meaning of subparagraph one (a).

Any bid hereafter made to any political subdivision of the state or any public department, agency or official thereof by a
corporate bidder for work or services performed or to be performed or goods sold or to be sold, where competitive
bidding is required by statute, rule, regulation or local law, and where such bid contains the certification referred to in
subdivision one of the section, shall be deemed to include the signing and submission of the bid and the inclusion as the
act and deed of the corporation.

Signed _____________________________   Title _______________________________




#### **APPENDIX B** **RESOLUTION - FOR CORPORATE BIDDERS ONLY**


RESOLVED THAT ______________________________________ be authorized to sign and submit the bid
or proposal of this corporation for the following project:


**Describe the project**

And to include in such bid or proposal the certificate as to non-collusion required by SECTION ONE
HUNDRED THREE – D of the General Municipal Law as the act and deed of such corporation and for any
inaccuracies or misstatements in such certificate this corporate bidder shall be liable under the penalties of
perjury.

The foregoing is a true and correct copy of the resolution adopted by;

Corporation at a meeting of its Board of Directors held on the __________ day of and is still in full force and
effect this __________ day of __________ 2025.

**__________________________________**

**(SEAL OF CORPORATION)**

**__________________________________**

**(SECRETARY)**




#### **APPENDIX C** **PROPOSER WARRANTIES**


A.​ Proposer warrants that it is willing and able to comply with State of New York laws and

regulations.

B.​ Proposer warrants that it is willing and able to obtain an error and omissions insurance policy

providing a prudent amount of coverage for the willful or negligent acts, or omissions of any
officers, employees or agents thereof.


C.​ Proposer warrants that it will not delegate or subcontract its responsibilities under an agreement

without the express prior written permission of the Edgemont Union Free School District.

D.​ Proposer warrants that all information provided by it in connection with this proposal is true and


accurate.

E.​ Proposer warrants that it has read the Request for Proposal (RFP) for Architect Services and fully

understands its intent and certifies that it has adequate personnel, equipment, and facilities to fulfill the
requirements thereto. It understands that their ability to meet the criteria and provide the required services
shall be reviewed by the District, which will develop a recommendation for the Board of Education’s
consideration regarding the selection of the most advantageous Architect Services proposal.

F​ Proposer certifies that all statements in this Affidavit and in any explanatory enclosures are, to the best of
its knowledge and belief, true and correct, and that any omission and/or misstatement of any material
fact(s) may cause the District to: (A) reject the submission of this proposal, (B) revoke any award of
contract from the Board of Education; and/or terminate the contract and any fees pending.

G.​ It is understood that all information included in, attached to, or required by this RFP shall become public

record upon delivery to the District. The undersigned certifies that the completion of the Proposal is a
binding commitment to provide the Architect Services requested as proposed herein.

**Firm’s Name**


**Address**


**City, State, Zip**


**(Print Name)​ ​** **(Signature)**


**(Phone)​** **(Fax)**




#### **APPENDIX D** **CERTIFICATION - IRAN DIVESTMENT ACT OF 2012**


**(THIS FORM MUST BE SIGNED AND NOTARIZED**SUBMIT WITH PROPOSAL)**


The Iran Divestment Act of 2012, effective as of April 12, 2012, is codified at State Finance Law (“SFL”)
§165-a and General Municipal Law (“GML”) §103-g. The Iran Divestment Act, with certain exceptions,
prohibits municipalities, including the District, from entering into contracts with persons engaged in
investment activities in the energy sector of Iran.

Each bidder or proposer must certify that it is not on the list of entities engaged in investment activities in Iran
created pursuant to paragraph (b) of subdivision 3 of Section 165-a of the SFL. In any case where the bidder
or proposer cannot certify that it is not on such a list, the bidder or proposer shall so state and shall furnish
with the bid or proposal a signed statement which sets forth in detail the reasons why such statement cannot
be made. The District may award a bid to a bidder who cannot make the certificate on a case by case basis if:

1.​ The investment activities in Iran were made before the effective date of this section (i.e., April 12, 2012),

the investment activities in Iran have not been expanded or renewed after the effective date of this section
and the person has adopted, publicized and is implementing a formal plan to cease the investment
activities in Iran and to refrain from engaging in any new investments in Iran; or

2.​ The District makes a determination that the goods or services are necessary for the District to perform its

functions and that, absent an exemption, the District would be unable to obtain the goods or services for
which the contract is offered. Such determination shall be made in writing and shall be a public document.


**A bid or proposal shall not be considered for award nor shall any award be made where the bidder or**
**proposer fails to submit a signed and verified bidder’s certification.**

Signature: ___________________________________________ Date: __________________

Affirmed to me this ________________ day of ____________________, 2025

Notary Signature: _____________________________________ Date: __________________




#### **APPENDIX E** **HOLD HARMLESS AND INDEMNIFICATION AGREEMENT**


**(THIS FORM MUST BE SIGNED AND NOTARIZED**SUBMIT WITH PROPOSAL)**

Architect agrees to indemnify and hold the Edgemont Union Free School District entirely harmless from
all liability arising out of:


1.​ Any and all claims under Workers’ Compensation acts and other employee benefit acts with respect

to Architect’s employees or Architect’s subcontractor’s employees arising out of Architect’s work
under this Agreement; and


2.​ If arising out of, pertaining to, or relating to the negligence, recklessness, or willful misconduct of

the Architect, the Architect shall indemnify and hold the Edgemont Union Free School District
harmless from any liability for damages for (1) death or bodily injury to person; (2) injury to, loss
or theft of property; (3) any failure or alleged failure to comply with any provision of law or (4) any
other loss, damage or expense arising under either (1), (2), or (3) above, other than as the result of
the negligent professional act or omission in the performance of professional services pursuant to
this Agreement, sustained by the Architect or the District, or any person, firm or corporation
employed by the Architect or the District upon or in connection with the Project, except for liability
resulting from the sole or active negligence, or willful misconduct of the District, its officers,
employees, agents or independent Architects who are directly employed by the District or of other
third parties not under the control or the supervision of the Architect.  The Architect, at its own
expense, cost, and risk, shall defend any and all claims, actions, suits, or other proceedings that may
be brought or instituted against the District, its officers, agents or employees, on any such claim or
liability, and shall pay or satisfy any judgment that may be rendered against the District, its officers,
agents or employees in any action, suit or other proceedings as a result thereof; and


3.​ If arising out of, pertaining to, or relating to the negligence, recklessness, or willful misconduct of

the Architect, the Architect shall indemnify and hold the District harmless from any loss, injury,
death to persons or damage to property caused by any negligent act, neglect, default or negligent
omission of the Architect, or any person, firm or corporation employed by the Architect, either
directly or by independent contract, including all damages due to loss or theft, sustained by any
person, firm or corporation including the District, arising out of, or in any way connected with the
Project, including injury or damage either on or off District property; but not for any loss, injury,
death or damages caused by the negligence, or willful misconduct of the District or of other third
parties not under the control or the supervision of the Architect. The Architect’s obligation to
indemnify does not include the obligation to defend actions or proceedings brought against the
District, but rather to reimburse the District for attorney’s fees and costs incurred by the District in
defending such actions or proceedings brought against the District.

Signature: ___________________________________________ Date: __________________

Affirmed to me this ________________ day of ____________________, 2025

Notary Signature: _____________________________________ Date: __________________




#### **APPENDIX F​** **PROPOSAL FORM**


|Soils|Day|Price|Unit|Comments|
|---|---|---|---|---|
|Inspect Sub-Grade|Half Day||||
|Inspect Soil Compaction|Half Day||||
|Inspect Fill Placement|Half Day||||
|On-site Soil Sampling|On-site Soil Sampling||ea||
|Sample Pick Up|Sample Pick Up||ea||
|Density Testing|Density Testing||ea||
|Proctor Testing|Proctor Testing||ea||
|Sieve Analysis|Sieve Analysis||ea||
|Percent Passing 200|Percent Passing 200||ea||
|Hydrometer Analysis|Hydrometer Analysis||ea||
|Backfill Material Analysis|Backfill Material Analysis||ea||
|Formal Reports / Daily|Formal Reports / Daily||ea||
|Formal Reports / Item testing|Formal Reports / Item testing||ea||
|Overtime - Regular Day|Overtime - Regular Day||hr||
|Overtime Saturday|Overtime Saturday||hr||
|Overtime Sunday and Holidays|Overtime Sunday and Holidays||hr||





|Concrete|Day|Price|Unit|Comments|
|---|---|---|---|---|
|Plant Inspection|Half Day||||
|Field Inspection|Half Day||||
|Formwork & Reinforcement Inspection|Half Day||||
|Slump Testing|Slump Testing||ea||
|Air Content Testing|Air Content Testing||ea||
|Unit Weight Testing|Unit Weight Testing||ea||
|Cast Test Cylinders|Cast Test Cylinders||ea||
|Cylinder / Sample Pickup|Cylinder / Sample Pickup||ea||
|Compressive Strength Testing|Compressive Strength Testing||ea||
|Concrete Mix Design|Concrete Mix Design||ea||
|Asphalt Density Testing|Asphalt Density Testing||ea||
|Formal Reports / Daily|Formal Reports / Daily||ea||
|Formal Reports / Item Testing|Formal Reports / Item Testing||ea||
|Concrete Slab Moisture Testing|Concrete Slab Moisture Testing||ea||
|Windsor Probe Testing Hardened Concrete|Windsor Probe Testing Hardened Concrete||ea||
|Overtime - Regular Day|Overtime - Regular Day||hr||
|Overtime Saturday|Overtime Saturday||hr||
|Overtime Sunday and Holidays|Overtime Sunday and Holidays||hr||





|Structural Steel|Day|Price|Unit|Comments|
|---|---|---|---|---|
|Shop Inspection|Half Day||||
|Inspection and Testing of Bolted Connections(Shop)|Inspection and Testing of Bolted Connections(Shop)||ea||
|Inspection and Testing of Welded Connections(Shop)|Inspection and Testing of Welded Connections(Shop)||ea||
|Inspection and Testing of Paint(Shop)|Inspection and Testing of Paint(Shop)||ea||
|Field Inspection|Half Day||||
|Field Inspection and Testing of Bolted Connections<br>(Field)|Field Inspection and Testing of Bolted Connections<br>(Field)||ea||
|Inspection and Testing of Welded Connections(Field)|Inspection and Testing of Welded Connections(Field)||ea||
|Inspection and Testing of Painting (Field)|Inspection and Testing of Painting (Field)||ea||
|Spray Fireproofing Inspection<br>Half Day|Spray Fireproofing Inspection<br>Half Day||||
|Full Day|Full Day||||
|Spray Fireproofing Density Testing|Spray Fireproofing Density Testing||ea||
|Spray Fireproofing Adhesion / Cohesion Testing|Spray Fireproofing Adhesion / Cohesion Testing||ea||
|Ultrasonic Testing|Ultrasonic Testing||Day||
|Magnetic Particle Testing|Magnetic Particle Testing||ea||
|Dye Penetrant Testing|Dye Penetrant Testing||ea||
|Formal Reports / Daily|Formal Reports / Daily||ea||
|Formal Reports / Item testing|Formal Reports / Item testing||ea||
|Overtime - Regular Day|Overtime - Regular Day||hr||
|Overtime Saturday|Overtime Saturday||hr||
|Overtime Sunday and Holidays|Overtime Sunday and Holidays||hr||





|Masonry|Day|Price|Unit|Comments|
|---|---|---|---|---|
|Field Inspections|Half Day||||
|Mortar / Grout Testing|Mortar / Grout Testing||ea||
|Sample Pickup|Sample Pickup||ea||
|Mortar / Grout Unit Weight|Mortar / Grout Unit Weight||ea||
|Compressive Strength Testing (Mortar /<br>Grout)|Compressive Strength Testing (Mortar /<br>Grout)||ea||
|Compressive Strength Testing|Compressive Strength Testing||ea||
|Water Retention Testing (Mortar / Grout)|Water Retention Testing (Mortar / Grout)||ea||
|Formal Reports / Daily|Formal Reports / Daily||ea||
|Formal Reports / Item testing|Formal Reports / Item testing||ea||
|Overtime - Regular Day|Overtime - Regular Day||hr||
|Overtime Saturday|Overtime Saturday||hr||
|Overtime Sunday and Holidays|Overtime Sunday and Holidays||hr||


|Firestopping|Day|Price|Unit|Comments|
|---|---|---|---|---|
|Field Inspections|Half Day||||
|Intumescent Paint Inspection<br>Half Day|Intumescent Paint Inspection<br>Half Day||||
|Full Day|Full Day||||
|Thickness and Adhesion Inspection|Thickness and Adhesion Inspection||ea||
|Formal Reports / Daily|Formal Reports / Daily||ea||




|Misc|Day|Price|Unit|Comments|
|---|---|---|---|---|
|PE|Half Day||||
|Meetings with PE|Meetings with PE||hr||
|Meetings withQualified Personnel|Meetings withQualified Personnel||hr||
|Formal Reports / Daily|Formal Reports / Daily||ea||
|Formal Reports / Item testing|Formal Reports / Item testing||ea||
|Travel Cost/ Mileage Cost|Travel Cost/ Mileage Cost||mile||



Note: Driving Miles are included in the unit price above


Firm’s Name


Address


(Print Name)​ (Signature)


(Phone)​ (Fax)







#### **APPENDIX G**


**SEXUAL HARASSMENT PREVENTION CERTIFICATION FORM**


By submission of this bid, the person signing on behalf of the bidder certifies, under penalty of
perjury, that: the bidder has and has implemented a written policy addressing sexual harassment
prevention in the workplace; the bidder provides annual sexual harassment prevention training to
all of its employees; and that the principal(s) and all employees of the bidder have completed the
sexual harassment prevention training in the last twelve (12) months. Such policy shall, at a
minimum, meet the requirements of Section 201-g of the Labor Law.


Bidder Name: __________________________________________________________________ JPC Logistics LLC dba JPCL Engineering LLC


Bidder Address: ________________________________________________________________ 2 Clerico Lane, Building #1, Hillsborough, NJ 08844


Print Name and Title: ____________________________________________________________ Asma Hashmi, Principal


Signature: _____________________________________________________________________


Date: _________________________________________________________________________ 05/27/2025



Sworn to before me this ______​27th
day of __________________, 20____ May 25



27th



May



25



______________________________


Notary Public



# File: RFP2.pdf

top five line counts: {'Telephone: ___________________________________Fax:___________________________________': 3, '- During the completion by the architect of the construction documents, update the Project': 2, 'bidding documents to be made available to bidders on the project.': 2, '- Establish and maintain an accurate and up-to-date construction cost accounting system.': 2, 'Address: ____________________________________________________________________________': 2}


**REQUEST FOR PROPOSALS**


**-------**

**PROPOSAL INSTRUCTIONS AND CONDITIONS**


**FOR**

**CONSTRUCTION MANAGEMENT SERVICES**


**FOR THE**

**EAST ISLIP UNION FREE SCHOOL DISTRICT**

The Board of Education, East Islip Union Free School District is issuing this Request for Proposals
(RFP) for the selection of a construction management firm to provide construction management
services in connection with various capital improvements to be undertaken by the School District
during the 2025-2026 school year and such other projects as the School District may request. The
School District has established a capital reserve for capital improvements at the District’s
facilities. The School District anticipates that an annual amount of up to $3,000,000 will be
deposited into the capital reserve for use in connection with capital projects subject to voter
approval.

The School District is seeking one or more qualified individuals/firms to provide construction
management services in connection with the various capital improvement projects that the School
District may undertake during the 2025-2026 school year and any future school years in which
the resultant contract is renewed. The Board of Education reserves the right to award one or more
contracts as a result of this RFP. Similarly, the Board of Education reserves the right to renew any
contract awarded pursuant to this RFP for additional one-year terms not to exceed three (3)
additional one-year terms.

Specifically, the individual (firm) selected as a result of this Request for Proposals will be expected
to perform his/her/its services in accordance with any or all of the terms of the agreement
annexed hereto as Exhibit “A”. As part of the RFP, Proposers should identify in detail the manner
in which the services to be required will be accomplished by him/her/it.

_**Any capital improvement project is subject**_ _**to the availability of District funds and the approval**_
_**of the voters of the District.**_

**I.** **DESCRIPTION OF THE SCHOOL DISTRICT**

The East Islip Union Free School District is located in Suffolk County and is comprised of six (6)
schools: the East Islip High School; East Islip Middle School; Timber Point Elementary School;
Ruth C. Kinney Elementary School; John F. Kennedy Elementary School; Connetquot Elementary
School; and the East Islip Union Free School District Administration Center.

At present, the District anticipates submitting approximately $3 million in proposed capital
improvements to its voters as part of the 2025-2026 school year budget.

**II.** **SCOPE OF SERVICES**

The Proposer shall provide the services set forth in the annexed agreement to assist the School
District in establishing the School District’s capital improvement program, the financial and time


requirements and limitations. The successful proposer’s services will include but not be limited

to:

1. **Pre-Bidding Services:**


  - Consult with the School District and the architect/engineer regarding the project
program, budget, timetable and advisability of various bidding options and timing of
construction.

  - Preparation of an independent estimate of the Project costs inclusive of the contingencies
based on the construction documents prepared by the School District’s
architect/engineer.

  - Preparation of a schedule that incorporates tasks of the entire team, during the Project’s
Design Phase and Construction Phase.

  - Presentations of the construction projects to be undertaken at public meetings to be held
in the community on an as-needed basis.

2. **General Responsibilities:**


  - Provide sufficient organization and experienced personnel and management to carry out
the requirements of this engagement. The Construction Manager will provide and
maintain a qualified, on-site field staff, with sufficient authority to act on the Construction
Manager’s behalf, to manage the project, conform to the scope of services, and insure that
the work is performed in compliance with the contract documents.

  - Provide administration management and related services necessary to coordinate the
activities of the Construction Manager, architect and contractors, as well as coordination
of the construction activities of the contractors with one another. The School District shall
have the right to interview and approve all of the Construction Manager’s personnel who
will be working on-site. The School District reserves the right to request the removal of
any of the Construction Manager’s personnel.

  - All employees assigned pursuant to the terms of this RFP are required to present a picture
identification before entering a school building, subject to background check and should
wear a visible form of identification while conducting business at all times on school
property.


3. **Scheduling:**


  - In coordination/cooperation with the School District’s architect/engineer, develop a
Project Master Schedule that establishes duration and responsibility for all major activities
during all Phases of the Project. The Project Master Schedule shall coordinate and
integrate the Construction Manager’s services, the architect’s services, Contractor’s
responsibilities and the School District’s responsibilities with regard to anticipated design
and construction.

  - Monitor the Project Master Schedule and report to the School District and
architect/engineer the progress during the pre-construction and pre-bid phases and will
notify the School District and the architect of any delays or problems and will recommend
any corrective action necessary to meet the time requirements for the project.

  - During the completion by the architect of the construction documents, update the Project


Master Schedule so as to develop a detailed Construction Schedule for inclusion in the
bidding documents to be made available to bidders on the project.

4. **Cost Estimating and Budget Analysis Services:**


  - Establish and maintain an accurate and up-to-date construction cost accounting system.

  - Prepare a minimum of three (3) detailed cost estimates for the building, construction and
site development work. One (1) estimate will be provided at the completion of schematic
design, one (1) at the completion of the design phase, and one (1) at the completion of
construction documents.

  - The Construction Manager will use capital projects software to maintain project records.

  - For each project undertaken during the term of the resultant contract, prepare a projected
project cashflow statement after contract award. The projected project cashflow statement
shall be updated and re-issued at least on a monthly basis or at such other interval as
determined or requested by the School District.

5. **Pre-Construction and Bidding Phase:**


  - Review construction documents and offer recommendations on construction and logistics
prior to establishing bid date.

  - Develop construction-phasing schedule to be included in contract documents.

  - Prepare a schedule that incorporates tasks of the entire team during the design phase and
the construction phase.

  - Monitor the project master schedule and report to the School District and architect, the
progress during the pre-bid phase and bidding phases, notify the School District and the
architect of any delays or problems and recommend any corrective action necessary to
meet the time requirements for the project.

  - During the completion by the architect of the construction documents, update the Project
Master Schedule so as to develop a detailed construction schedule for inclusion in the
bidding documents to be made available to bidders on the project.

  - Establish and maintain an accurate and up-to-date construction cost accounting system.

  - Prepare detailed cost estimates at various intervals of the project beginning at the
completion of the schematic design through completion of construction documents.

  - Assist the Design Firm with local labor market conditions and availability of construction
materials as they affect design choices and construction costs.

  - Undertake value engineering studies of alternative systems, with respect to
manufacturers and vendors of items with long lead times, to obtain the most economic
benefit from a cost standpoint while maintaining the School District’s functional and
program requirements.

  - Assist the architect/engineer in developing a multiple prime bid strategy. Review the bid
packages as prepared by the architect/engineer for each trade including scope definition,
terms and conditions, insurance and bonding requirements. Conduct pre-bid conferences,
solicit local bidder interest for this project and maximize competition to achieve the best
bidding results for the School District.

  - Assist the School District and architect/engineer in evaluating the responsiveness and
completeness of construction bids received.


  - Review and make recommendations to award contracts to successful bidders.

6. **Construction Phase and Closeout**


  - Assist the School District in selecting and coordinating the professional services of
surveyors, special consultants and testing laboratories required for the project.

  - Presence at pre-construction conferences, job meetings, and special meetings as necessary,
to be attended also by the architect/engineer, the School District, and/or the Contractors
performing the work on the project.

  - Establish and maintain an on-site office in office space provided by the School District.

  - Provide Construction Project Manager(s) to oversee and coordinate contractors, to
expedite their work and maintain quality control and conformance to the contract
documents. The construction inspection services shall be thorough, examining all
construction work for conformance to the design plans and specifications, performed on
a daily basis for all trades and for each construction project.

  - The Construction Project Manager(s) shall review and fully comprehend the scope of
work outlined in the contract bid documents, drawings and specifications. Copies of the
Contract Bid Documents shall be maintained at the site and be made available for
referencing by the architect/engineer and School District and be used by the Construction
Management Team. The Construction Project Manager shall prepare and maintain a
current master record copy of drawings showing all changes to the contract drawings.

**III. INSURANCE**

The Construction Manager shall provide Workers’ Compensation, automobile, comprehensive
general liability and professional liability (Errors and Omission) insurance in amounts set forth
herein. All policies shall be in a form, with a deductible, and with a New York State licensed and
admitted carrier. Promptly upon signing the construction management services agreement, and
thereafter as necessary or requested, the Construction Manager shall provide the School District
with proof of the required insurance coverages. Simultaneously with the execution of this
Agreement, Construction Manager shall deliver insurance policies and certificates to School
District which will provide at least a 30-day notice of cancellation or amendment.

_**Required Insurance:**_

1. Minimum Required Insurance:
**a.** **Commercial General Liability Insurance**

$1,000,000 per Occurrence/ $2,000,000 Aggregate with coverage for sexual
misconduct
$2,000,000 Products and Completed Operations
$1,000,000 Personal and Advertising Injury
$100,000 Fire Damage
$10,000 Medical Expense

**b.** **Automobile Liability**

$1,000,000 combined single limit for owned, hired, borrowed and non-owned motor
vehicles.


**c.** **Workers' Compensation and NYS Disability Insurance**

Statutory Workers' Compensation (C-105.2 or U-26.3); and NYS Disability Insurance
(DB- 120.1) for all employees. Proof of coverage must be on the approved specific
form, as required by the New York State Workers' Compensation Board. ACORD
certificates are not acceptable. A person seeking an exemption must file a CE-200 Form
with the state. The form can be completed and submitted directly to the WC Board
online.

**d.** **Professional Errors and Omissions Insurance**

$2,000,000 per occurrence/$2,000,000 aggregate for the professional acts of the
consultant performed under the contract for the District. If written on a “claims-made”
basis, the effective date must pre-date the inception of the contract or agreement.
Coverage shall remain in effect for three years following the completion of work.

**e.** **Umbrella/Excess Insurance**

$3,000,000 each Occurrence and Aggregate. Umbrella/Excess coverage shall be on a
follow-form basis or provide broader coverage over the required Auto Liability
(where applicable), General Liability and Professional Liability coverages.

The Construction Manager acknowledges that failure to obtain such insurance on behalf of the
School District constitutes a material breach of contract and subjects it to liability for damages,
indemnification and all legal remedies available to the School District. The Construction Manager
is to provide the School District with a certificate of insurance, evidencing the above requirements
have been met, prior to the provision of services. The failure of the School District to object to the
contents of the certificate or the absence of same shall not be deemed a waiver of any rights held
by the School District.

2. Notwithstanding any terms, conditions or provisions, in any other writing between the
parties, the professional consultant hereby agrees to effectuate the naming of the School
District as an additional insured on the professional consultant's insurance policies, except
for workers' compensation and N.Y. State Disability insurance.

**IV. THE SELECTION PROCESS**

**A.** **Timetable**


The School District is expected to undertake the selection process according to the
following schedule:


Deadline for submission May 15, 2025 at 10:00 a.m.
of proposals:

Interview with finalists: The School District anticipates that
interviews of the finalists will be held on
May 19, 2025. Dates and times for interviews
shall be finalized after submission of


proposals. Interviews will be conducted by
the Board of Education and District

Administration.

Selection of firm: upon Board of Education approval on or
about June 10, 2025


Contract review and execution: to be scheduled with the School District’s
legal counsel following selection of firm by
the Board of Education

**B.** **Site Visits**


An informational meeting/site visit will be held on April 29, 2025 at 10:00 a.m. Potential
respondents are asked to gather at the District’s Administrative Offices, East Islip Union
Free School District, 1 Craig B. Gariepy Avenue, Islip Terrace, New York 11752 at which
time they will be escorted to the sites of possible future renovation and repair work, as
determined by the School District. It is highly recommended that all potential respondents
attend. Failure to attend this meeting does not absolve the respondent from compliance
with all terms and conditions of the RFP or any resultant agreement. Respondents may
also visit the buildings by making prior arrangements with Jenny Bejarano, Purchasing
Agent at 631-224-2031. No one may visit the building or site without an appointment.

**C.** **Proposal Evaluation Criteria**

Proposals will be evaluated on the basis of the following criteria:


1. Experience and Qualifications of the Proposer: Consideration will be given to
proposers demonstrating strong capabilities, experience and reputation in
undertakings similar to those described in this RFP, and providing authoritative
documentation of their financial soundness and stability. Similar experience will
be understood to include, but not be limited to:


a. At least ten (10) years’ experience in the provision of construction
management services on projects involving school district renovations,
alterations and/or additions;

b. At least ten (10) projects in school district expansions, renovations and
upgrades.


2. Ability to Implement District Projects Promptly: Preference will be given to
proposals demonstrating an ability to carry out the tasks and responsibilities
outlined in the proposal in a prompt and efficient manner. The capability of the
construction management firm to progress and complete the work will be
evaluated taking into consideration size of work force, current workload, in house
staff capabilities and the like.


3. Level of client satisfaction determined from supplied references.

4. Logistics and familiarity with the geographic area.

5. Experience and qualifications of the professional staff to be assigned to this project.

6. Proposed methodology and demonstrated understanding of the project
requirements and School District needs, including the need for a team effort with
the School District and other construction professionals.

7. Demonstrated knowledge of:


a. New York State Education Department Project Application and Approval

Process


b. New York State Education Department Office of Facilities & Management

Services Forms


c. Application and Certificate for Payment (Form AIA Document G702CMa)


d. Green Building Design and New York State Education Department CHPS
knowledge and experience.


e. New York State Education Department Change Order Process.


8. Fee for services to be provided in a separate sealed envelope marked “Fee
Proposal”.

9. Insurance: Ability to provide the insurance required in the attached sample
agreement.


10. Pending Litigation and Insurance Claims: Provide a list of all pending litigation
against Proposer. Also provide a list of any pending insurance claims filed with
Proposer’s current or previous insurance carrier and the nature of such claims.

11. References

**V. RFP PROCEDURES**

**A.** **Information and Documents**


Questions concerning the RFP and the procedures for responding should be directed in
writing to Jenny Bejarano, Purchasing Agent at [jenny.bejarano@eischools.org. Questions,](mailto:jenny.bejarano@eischools.org)
in writing, must be received no later than at 2:00 p.m. on May 2, 2025. Responses to
questions posed will be answered in writing and distributed to all prospective Proposers.


**B.** **Submission of Proposals**


Respondents should submit an original and five (5) copies of their proposal. Respondents
should also submit an original and five (5) copy of their fee proposal in a sealed envelope
that is clearly marked “Fee Proposal”. Proposals must be received no later than 10:00 a.m.
on May 15, 2025 at the District’s Administrative Offices, East Islip Union Free School
District, 1 Craig B. Gariepy Avenue, Islip Terrace, New York 11752. To prevent opening
by unauthorized individuals, your proposal should be clearly identified on the envelope
wrapper as follows: “CONSTRUCTION MANAGEMENT RFP” and should be addressed
to Jenny Bejarano, Purchasing Agent, East Islip Union Free School District.

**C.** **Modification or Withdrawal of Proposals**


Any proposal may be withdrawn or modified by written request of the Proposer,
provided that such request is received by the Purchasing Agent at the above address prior
to the date and time set for receipt of proposals.

**D.** **Right to Reject Proposals**

This RFP does not commit the School District to award a contract, pay any cost incurred
in the preparation of a proposal in response to this RFP, or to procure or contract for
services. The School District intends to award a contract on the basis of the best interest
and advantage to the School District, and reserves the right to accept or reject any or all
proposals received as a result of this RFP, to negotiate with all qualified proposers to
request additional information or clarification from any firm that submitted a valid
proposal, or to cancel this RFP in part or in its entirety, if it is in the best interest of the
School District to do so. The School District may select as the successful proposer that
proposal which, in the School District’s sole discretion and with whatever modifications
the School District and the proposer may mutually agree upon, best meets the School
District’s requirements whether or not that proposal is the lowest priced. No proposer
shall have any legal, equitable, or contractual rights of any kind arising out of its
submission of a proposal except as and to the extent that the School District, in its sole
discretion, shall enter into a contract with the proposer that it selects as the successful

proposer.

**VI. PROPOSAL FORMAT AND CONTENTS**

Proposals must be submitted in the format outlined in this section. Each of the described parts
and sections must be completed in full (except those sections described as optional). Each
proposal will be reviewed to determine if it is complete prior to actual evaluation. The School
District reserves the right to eliminate from further consideration any proposal deemed to be nonresponsive to the requests for information contained herein. Each of the parts and sections
described below should begin on a separate page, and each page should state the name of the
proposer in the upper right hand corner.


**A.** **Proposer Background and Qualifications**


Section A of the proposal must contain the following information about the proposer and
any independent contractor with which the proposer intends to work on the project.


FORM I: Form I should describe the overall make-up of the project team, and a
detailed statement as to how the Proposer intends to meet the needs of the
School District. Proposers should include a chart depicting the
management structure envisioned for the projects. Include the size of the
firm and the location of the office from which the work of this contract is
to be performed. Provide a listing of all in-house services provided (i.e.
architectural, engineering, site planning, surveying, environmental, etc.).

FORM II: Form II should provide information about the Proposer and each
independent contractor/consultant to be used by the successful Proposer
to provide the services to the School District, if applicable. (Addresses,
telephone/fax numbers, names of contact person and lead person,
qualifications licenses and statement of insurance coverage.) Individual
résumé for each of the individuals on the project team must be included in
this section.

FORM III: Describe the prior relevant experience of the Proposer and members of the
project team, and independent contractors, if any. For each include the
information listed below:


1. Customer’s name.

2. Total project capital cost.

3. Type of contract.

4. Name & telephone number of reference for the project.

5. Brief description of the project’s scope of services and status
(including type of facility at which project was implemented,
whether the project was timely completed and whether significant
problems occurred that affected project performance). The right to
call the reference and/or visit the project sites provided by the
Proposer will be presumed by the School District.

6. Photographs or video of any projects in school districts including
expansion projects, renovations projects and upgrade projects.

FORM IV: (optional) include any additional information about the project team, its
personnel, financial condition, or qualifications regarded as being
pertinent.


**B.** **Financial Background Information**


Section B of the RFP must contain detailed information concerning the financial
background of the Proposer and any independent contractors to be engaged by the
Proposer, including but not limited to financial statements, annual reports and the like
covering the most recent fiscal year for the proposer.

**C.** **Form of Proposal**


Proposals submitted pursuant to this RFP shall include the Proposer’s cost to perform the
services set forth herein. Said cost shall be a structured as a sliding scale based on a
percentage of the construction cost, to be paid to the successful proposer in accordance
with the terms of the resultant Contract and shall be submitted as FORM V of this RFP in
a separate sealed envelope marked “Fee Proposal.”

**D.** **Form of Agreement**


The School District does not intend to use an AIA Standard Form contract in connection
with the services to be provided herein. The terms of said agreement are annexed to this
RFP. **The School District reserves the right to amend the terms of the attached**
**agreement.**


**VII. CONSTRUCTION MANAGER INFORMATION FORM**

Name of firm, address, and contact information for your firm’s main office:

Firm: _______________________________________________________________________________

Address: ____________________________________________________________________________

City, State, Zip: ______________________________________________________________________

Telephone: ___________________________________Fax:___________________________________

Email Address:______________________________________________________________________

Name, title, address, and contact information for the firm’s officer responsible to the District for
all the work to be provided under this RFP:

Name/Title: _________________________________________________________________________

Address: ____________________________________________________________________________

City, State, Zip: ______________________________________________________________________

Telephone: ___________________________________Fax:___________________________________

Email Address:_______________________________________________________________________


Please check the appropriate box to identify the legal entity making this proposal.

( ) Corporation ( ) Partnership ( ) Individual ( ) Joint Venture


If a Joint Venture, identify other firm(s) in association.

Firm/Contact:_______________________________________________________________________

Address:____________________________________________________________________________

City, State, Zip:______________________________________________________________________

Telephone: ___________________________________Fax:___________________________________

Email Address:_______________________________________________________________________


**If yes to any of the following questions, please attach a separate sheet of explanations; state**
**the project(s), date, owner and reason.**

Has your firm or joint venture failed to accept a contract or to complete a contract awarded to
you? ( ) Yes ( ) No

Has your firm or joint venture under another or previous name failed to accept a contract or to
complete a contract awarded to you? ( ) Yes ( ) No

Has your firm or joint venture had a contract terminated or been given written notice or
demand incident to a proposed contract termination? ( ) Yes ( ) No

State the name, agent’s name, address, telephone number of your current bonding company(s)
and identify any other sureties used by your firm during the past five (5) years.


Company Name ________________________________________________________

Agent Name _______________________________________________________


Address _______________________________________________________
City, State, Zip _______________________________________________________

Telephone ___________________________________Fax _________________


**PROPOSAL FORM FOR CONSTRUCTION MANAGEMENT SERVICES**

_**Fee Proposal for Construction Management Services**_ :

The School District is requesting proposals for the 2025-2026 school year on a sliding scale
percentage based on the amount approved by the voters for the School District’s capital
improvement program for the 2025-2026 school year and any renewals. Percentage fees will be
applied to the construction costs for projects approved by the District’s voters. The fee is subject
to adjustments based on the actual construction costs.


  - Fee based on voter authorization ranging from $0-$3,000,000 _______%

  - Fee based on voter authorization ranging from $3,000,001-$6,000,000 _______%

  - Fee based on voter authorization ranging from $6,000,001-$10,000,000 _______%

  - Fee based on voter authorization over $10,000,000 _______%

_**In addition**_ :

Proposers must include, as a separate line item in their proposals, the cost for provision of a trailer
for the Construction Manager’s use. Should the District elect to provide office space to the
Construction Manager, such sum shall be credited to the District.

Proposers will be compensated for the provision of additional services on an hourly basis. Each
proposer shall submit an hourly rate schedule for this purpose as part of its fee proposal. All
hourly rate schedules must include the position and corresponding rate for each team member.



# File: RFP1.PDF

top five line counts: {'```': 2, '- Visual inspection for health and safety concerns: Mechanical Heating and Cooling Equipment:': 2, 'exterior': 2, '**Respondent Q&A Zoom** : Tuesday April 1, 2025 9am': 1, '=': 1}


**Respondent Q&A Zoom** : Tuesday April 1, 2025 9am


=
[https://us02web.zoom.us/j/85361672810?pwd](https://us02web.zoom.us/j/85361672810?pwd=cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1) cj3FMhWgTSxYy9zsMbo3GLM1xx78Jy.1


**RFP General Information contact** [: LLR-Support@bedford2030.org](mailto:LLR-Support@bedford2030.org) or call (646) 389-6749


**Submission via email only** : townclerk@bedfordny.gov


________________________________________________________________


**I. PURPOSE**


This Request for Proposal (RFP) seeks responses from qualified NYSERDA-approved third-party
energy auditors to complete health and safety energy audits on multi-family rental buildings in the Town
of Bedford (the “Town”). Successful respondents will be included in a Town-approved list from which
landlords will select and engage directly with the Town of Bedford Building Inspector and other Town
personnel to perform inspections of certain multi-family rental residences located in the Town. These
audits are required by the Town’s Landlord Registry and Rental Property Health and Performance Law
(the “Law”) and must be conducted on qualified registered rental buildings (defined as 2+ unit rental
properties occupied by unrelated persons) every five (5) years. The Law also requires public posting of
inspection results and inclusion of information in building energy reports.


Successful respondents must participate in the NYSERDA Residential Energy Assessment Program and
must be qualified to inspect buildings for violations of specific New York State Building Code and Fire
Code requirements and other health and safety criteria (“Health & Safety Inspections and Audits”). The
Health & Safety Inspections & Audits will operate in conjunction with NYSERDA’S home energy
efficiency programs - EmPower+ and Residential Energy Assessment (REA), in order to identify energy
performance improvements and check for New York State Building Code and Fire Code requirements.
Both programs provide no-cost energy audits and income-eligible community members can apply for
Empower+ to receive discounts on energy efficiency upgrades following the audit. It is important for
inspectors to share information on tax credits and rebates with building owners so that owners are well
informed about costs and benefits of building upgrades.


Respondents who fulfill these requirements and provide a competitive fee to landlords will be given
priority consideration.


**II. BACKGROUND**


The Town of Bedford is located in Northern Westchester County with a population of approximately
17,500 residents and is nearly forty square miles. The Town has approximately 1,300 residential rental
units meeting the criteria for the Health & Safety Inspections and Audits. Widely regarded as a climate
action leader in New York State, the establishment of the Law and associated Health & Safety
Inspections and Audits, and the benchmarking of data obtained during such Health & Safety Inspections
and Audits, will allow Town to continue to position itself as a leader with respect to climate action
issues. While the genesis of these Health & Safety Inspections and Audits is primarily to ensure health
and safety criteria are met, the Town’s goal is to bolster its reputation as a leader in climate issues and as
an attractive and sought after community by making benchmarking data from the Health & Safety
Inspections and Audits publicly available.


**III. PROCESS/SCOPE OF WORK & DELIVERABLES**


**Inclusion on Town Approved List**


Successful respondents will be included on an approved Town list of inspectors from which landlords
will engage directly to arrange for Health & Safety Inspections and Audits of qualified rental properties.
In addition to your name and contact information, the Town will include information on its website with
information about the Law, steps that need to be taken for compliance, and the approximate out of
pocket costs for the Health & Safety Inspections and Audits.


Note: In cases where the building qualifies for an EmPower+ inspection the Town may provide funds to
supplement fees above the NYSERDA incentives using grant funding. Priority for such incentives will
be given to buildings located in the Bedford Hills community zip codes (10507) as they are a
NYSERDA previously disadvantaged community (DAC) and currently considered a New York State
Department of Environmental Conservation Potential Environmental Justice Area (PEJA).


**Process for Health & Safety Inspections and Audits**

1. **Scheduling.** Building owners will schedule an Health & Safety Inspections and Audits directly
with the inspection company from the approved list consisting of successful respondents to this

RFP.

2. **Inspections.** Inspector conducts Health & Safety Inspections and Audits (criteria listed in the
next section) and logs results into a form via an online app called Municity. Municity is the
Town’s Building Management System. Owners and prospective renters will be able to view data
from Municity. Results from the Health & Safety Inspections and Audits will have to be
submitted through both the normal NYSERDA process (EmPower or REA) for reimbursement,
as well as into the Town’s online system - Municity. The Municity system can be accessed
remotely and online.
3. **Benchmarking.** Inspectors that are able to assist in facilitating the collection of energy bills to
be added to Portfolio Manager and set up a building’s Portfolio Manager Account with
the/building owner to allow Benchmarking of particular properties will be favored.


Benchmarking is the process of measuring a building’s energy and water use by utilizing the
EPA’s Portfolio Manager online software, which allows tracking and comparing energy use over

time.

4. **Billing and Payment:** Qualified Inspectors will submit their bill to NYSERDA through the
normal NYSERDA reimbursement process. Any additional cost above the NYSERDA
reimbursement will be billed to the owner of the building with details listed below:


a. For:


i. An EmPower audit: the inspector will bill any additional costs, above any
NYSERDA reimbursement to the building owner with a copy to the Town. The
building owner will be responsible for payment directly to the inspector. The
Town may pay the contractor up to 100% of the additional cost on behalf of the
building owner if reimbursement funds are available from a grant the Town

received from NYSERDA.

ii. Market rate audit: The inspector will submit a bill for the additional cost above
any NYSERDA reimbursement, to the building owner with a copy for the Town.
The building owner will be responsible for payment. The Town may pay a
portion of the additional cost to the contractor on behalf of the building owner if
reimbursement funds are available from a grant the Town received from

NYSERDA.

iii. Grant funds will be distributed for the first round of inspections to pay for Health
& Safety Inspections and Audits in whole or in part prioritizing EmPower audits
and Market Rate audits on buildings housing low and moderate income residents.
Once grant funds are exhausted they will not be replenished and future Health &
Safety Inspections and Audits will be paid fully out of pocket for all building

owners.

5. **Failed Inspections.** For any buildings that fail a Health & Safety Inspection and Audit, the
Inspector will be required to report compliance or non-compliance to the Town Building
Department and the following procedures will apply:

i. The Town will require building owners to schedule re-inspections with an
Inspector within 30 days after receipt of the notice of failure.
ii. The inspector will be required to provide an affidavit to be submitted to support
any re-inspection of the property.
iii. The cost of the re-inspection should be disclosed to the building owner, and will
be paid by the building owner.
iv. The Town Building Department may issue a Summons after 60 days of inaction
based on the affidavit provided by inspector.
v. If asbestos is found in the dwelling unit the inspector will notify the appropriate
New York State/Westchester County entity.
vi. If lead is found in the dwelling unit the inspector will notify the appropriate New
York State/Westchester County entity.


**IV. SUBMISSION REQUIREMENTS**


Please provide detailed responses to the following:


**1.** **Company Overview:**

`o` Company name, address, and contact information.
`o` A brief overview of your company’s history and experience in conducting building
inspections.

**2.** **Qualifications and Certifications:**

`o` List of certifications and licenses held by your inspectors (e.g., ICC certifications, New
York State/local licenses). Inspectors will be required to provide annual recertification
documentation.
`o` Experience of inspectors with similar projects and multi-family rental building types.
`o` List of NYSERDA and/or utility inspections/audits your inspectors are certified to
complete.

`o` List of programs NYSERDA you participate in.

`o` Are you NYS Code Certified to perform code portions of the inspections?
**3.** **Scope of Services:**

`o` Detailed description of inspection services you offer.
`o` List of specific types of inspections you are qualified to perform (e.g., structural,
electrical, plumbing, fire safety).

**4.** **Experience:**

`o` Provide examples of recent projects similar in scope and size.
`o` References from previous clients with similar inspection needs.
`o` Experience with the use of Municity?

`o` Do you provide support to clients for Portfolio Manager account setup and energy data
uploads?

`o` Are you experienced in assisting customers establish a Portfolio Manager account?

`o` Are you experienced in assisting customers link/share their building’s Portfolio Manager
account to their municipalities’ account?

`o` Do you assist customers source/understand their utility bills?

`o` Do you assist customers with entering utility bills into Portfolio Manager?

`o` If you are an EmPower contractor, do you provide support for clients to navigate the
income qualification requirements for EmPower audits?

`o` How long have you been an REA and/or REA contractor? Please indicate length of time
for each program if you participate in both.

```
     o

```

**5.** **Inspection Process:**

`o` Describe your typical inspection process, including pre-inspection, inspection, and postinspection procedures.
`o` Outline any software or tools used for inspections and reporting.

**6.** **Customer Service & Scheduling and Availability:**

`o` Typical response time for scheduling inspections.


`o` Availability of inspectors and flexibility in scheduling.
`o` Please share your approach and process for customer service inquiries.

`o` Identification of the entity/firm's professional staff members who will be assigned to this
engagement if the entity/firm's Proposal is selected.

**7.** **Cost Structure:**

`o` Provide the fee to inspect for the Health & Safety Inspections and Audits (listed above)
in addition to the standard NYSERDA inspection requirements, if any?

`o` Provide the fee for a reinspection after a failed first inspection?

`o` Does your fee differ between REA and EmPower audits?

`o` Provide a general overview of your pricing structure.
`o` Provide discounts provided for multiple unit inspections, if any.
`o` Provide any additional costs or fees that may be associated with the inspection services.

**8.** **Insurance and Bonding:**

`o` Provide proof of insurance and bonding.
`o` Types of coverage and limits. (Town typical insurance requirements are attached as
Exhibit “A”).
`o` Upon selection in order to proceed with Health & Safety Inspections and Audits, you will
be required to provide Certificates of Insurance naming the Town as an additional
insured.

**9.** **Compliance:**

`o` How do you ensure compliance with local building codes and regulations?
`o` Any recent updates or training related to local building codes.

**10.** **Capacity:**


`o` The Town’s rental building stock will require 2,000 unit inspections within 450 buildings
in an 18 month period. Provide an estimate regarding how many Health & Safety
Inspections and Audits could your firm complete over 18 months?

**11.** **Indemnification and Hold Harmless**


`o` Upon selection all respondents will need to execute an Indemnification and Hold
Harmless Agreement in the form attached hereto as Exhibit “B”.


**12.** **Independent Status**


`o` Respondent will contract directly with building owners and shall not enter into a
contractual relationship with the Town. The respondent, its employees, officers, agents,
vendors, and sub vendors shall be considered to be independent contractors and shall not
claim or receive any benefit or privilege conferred to the Town’s officers, employees,


agents, or representatives, including, without limitation disability or unemployment
insurances, workers’ compensation, medical insurance, sick leave, retirement benefits or
any other employment benefit conferred upon the Town’s employees, officers, agents, or
representatives. The Town shall not be responsible for respondent’s compliance with any
local, state or federal law, regulation or requirement. Respondent assumes responsibility
for any and all acts and omissions of its employees, associates, agents, officers,
subcontractors and sub vendors as they relate to the services/work to be provided in
response to this Proposal.


**V. CONTRACTOR CONTACT**


If you have any questions or need additional information, please contact [Insert Your Name] at [Your
Contact Information].


**Submitted By:**

[Your Name]

[Your Title]

[Your Company]

[Contact Information]


**Signature:**

[Your Signature]

[Date]


**Submission via email** : townclerk@bedfordny.gov

# **VI. HEALTH AND SAFETY CRITERIA**


**New York State Building Code and Fire Code:**
**All NYS components are Pass/ Fail-**

 - Health & Safety Conditions Requiring Remediation: Fuel gas leaks

 - Health & Safety Conditions Requiring Remediation: Poorly vented heating appliances

 - Health & Safety Conditions Requiring Remediation: Puddling of fuel liquids

 - Health & Safety Conditions Requiring Remediation: Friable Asbestos

 - Health & Safety Conditions Requiring Remediation: Paint in poor conditions in buildings built

before 1979

 - Visual Inspection for Safety Criteria: No exposed and/or overloaded electrical outlets

 - Visual Inspection for Safety Criteria: Damaged walls, roof or flooring

 - Visual Inspection for Health & Safety Concerns: Heat and hot water operational

 - Visual Inspection for Safety Criteria: Smoke & Carbon Monoxide Alarms to be installed
throughout the dwelling unit per NY State Property Maintenance Code requirements

 - Visual Inspection for Safety Criteria: Fire extinguishers

 - Visual Inspection for Safety Criteria: Exit lighting

 - Visual Inspection for Safety Criteria: Stairwells of 4 risers or more must have handrails


**Local Health and Safety Criteria:**

 - Visual inspection for health and safety concerns: Mechanical Heating and Cooling Equipment:
Recordation of fuel type, approximate age and efficiencies

 - Visual inspection for health and safety concerns: Mechanical Heating and Cooling Equipment:
Distribution system

 - Visual inspection for health and safety concerns: Ducting inspection - filtration; leaks - unsafe
returns, major leaks, no filtration

 - Visual Inspection for Health & Safety Concerns: Oil Tank Issues

 - Visual Inspection for Health & Safety Concerns: Moisture: Visual bio growth

 - Visual Inspection for Health & Safety Concerns: Moisture: Relative humidity above 60%

 - Visual Inspection for Health & Safety Concerns: Moisture: Evidence of puddling of water /

water intrusion

 - Visual Inspection for Health & Safety Concerns: Moisture: Separation of conditioned and
unconditioned spaces

 - Indoor Air Quality Test: Particulate matter

 - Indoor Air Quality Test: Elevated CO2 Levels

 - Visual Inspection for Health & Safety Criteria: Existing Building Conditions: Stove Venting to

exterior

 - Visual Inspection for Health & Safety Criteria: Existing Building Conditions: Bath Venting to

exterior

 - Visual inspection for health and safety concerns: Existing Building Conditions: Vermiculite in

attics

 - Visual inspection for health and safety concerns: Existing Building Conditions: Knob and Tube
wiring

**Pass/Fail:**

 - Indoor Air Quality Test: Elevated CO Levels


**CVFG@GR�?**


EXHIBIT B

